# Lesson 1: Router Engine

## SETUP

In [1]:
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
import nest_asyncio

## LOAD DATA

In [3]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["../files/metagpt.pdf"]).load_data()
documents[:2]

[Document(id_='2beb6959-9b90-4943-a50f-a83c2c3e738e', embedding=None, metadata={'page_label': '1', 'file_name': 'metagpt.pdf', 'file_path': '../files/metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-06-08', 'last_modified_date': '2024-06-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Preprint\nMETAGPT: M ETA PROGRAMMING FOR A\nMULTI -AGENT COLLABORATIVE FRAMEWORK\nSirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,\nCeyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,\nLiyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8\n1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,\n3Xiamen University,4The Chinese Univ

## DEFINE LLMS

In [4]:
# define llms - OPENAI
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [5]:
# define llms - OLLAMA

from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="llama3:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

# pass_embedding = ollama_embedding.get_text_embedding_batch(
#     ["This is a passage!", "This is another passage"], show_progress=True
# )
# print(pass_embedding)

# query_embedding = ollama_embedding.get_query_embedding("Where is blue?")
# print(query_embedding)

## GET EMBEDDINGS

In [6]:
# chunk documents into sentence nodes
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents=documents)

In [7]:
from llama_index.core import SummaryIndex, VectorStoreIndex  # index -> Embeddings

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## DEFINE QUERY ENGINE & SET METADATA

In [8]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [9]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=("Useful for summarization questions related to MetaGPT"),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=("Useful for retrieving specific context from MetaGPT paper."),
)

## ROUTER ENGINE

In [12]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
import nest_asyncio

nest_asyncio.apply()

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True,
)

In [14]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: This choice indicates that the document is useful for summarization questions related to MetaGPT..
The document introduces MetaGPT, a meta-programming framework that enhances the problem-solving capabilities of multi-agent systems based on Large Language Models (LLMs) by incorporating human-like Standard Operating Procedures (SOPs). MetaGPT assigns specific roles to agents, streamlines workflows, and improves communication efficiency. It focuses on role specialization, workflow organization, structured communication interfaces, and executable feedback mechanisms to enhance code generation quality during runtime. Through extensive experiments, MetaGPT demonstrates superior performance on various benchmarks, outperforming existing approaches. The document also discusses the development process using MetaGPT for software projects, highlighting the structured approach from user input commands to system design. Additionally, it delves into the performance of differ

In [16]:
print(query_engine.query("get me the list of all sections of this document"))

Selecting query engine 1: This choice is more relevant as it specifically mentions retrieving specific context from the MetaGPT paper, which aligns with the request to get the list of all sections of the document..
The sections of this document are as follows:
1. Figure 12: The program call flow for “recommendation engine development” generated by the architect agent
2. Figure 3: A diagram showing the software development process in MetaGPT
3. Communication Protocol
